In [1]:
%%writefile instruct.yaml 

file_type: csv
dataset_name: custom_1988_2020
file_name: custom_1988_2020
table_name: 103
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns:
    - '198801'
    - '1'
    - '103'
    - '100'
    - 000000190
    - '0'
    - '35843'
    - '34353'

Overwriting instruct.yaml


In [2]:
%%writefile testutility.py 

import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re


################
# File Reading #
################

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string

def col_header_val(df,table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: str(x).lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: str(x).lower(), list(df.columns)))
    df = df.reset_index()
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Overwriting testutility.py


In [4]:
# Read config file
import testutility as util
config_data = util.read_config_file("instruct.yaml")

In [5]:
# dictionary with which you can subset whichever key you want
config_data['inbound_delimiter']

','

In [6]:
#inspecting data of config file
config_data

{'file_type': 'csv',
 'dataset_name': 'custom_1988_2020',
 'file_name': 'custom_1988_2020',
 'table_name': 103,
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': ['198801', '1', '103', '100', '000000190', '0', '35843', '34353']}

In [7]:
import dask.dataframe as dd

In [8]:
# read the file using config file
file_type = config_data['file_type']
source_file = config_data['file_name'] + f'.{file_type}'
df = dd.read_csv(source_file)
df.head()

,198801,1,103,100,000000190,0,35843,34353
0,198801,1,103,100,120991000,0,1590,4154
1,198801,1,103,100,210390900,0,4500,2565
2,198801,1,103,100,220890200,0,3000,757
3,198801,1,103,100,240220000,0,26000,40668
4,198801,1,103,100,250410000,0,5,8070


In [14]:
# number of pandas dataframes in a single dask dataframe
df.npartitions

71

In [10]:
#validate the header of the file
util.col_header_val(df,config_data)

column name and column length validation failed
Following File columns are not in the YAML file ['index']
Following YAML columns are not in the file uploaded []


0

In [11]:
print("columns of files are:" ,df.columns)
print("columns of YAML are:" ,config_data['columns'])

columns of files are: Index(['198801', '1', '103', '100', '000000190', '0', '35843', '34353'], dtype='object')
columns of YAML are: ['198801', '1', '103', '100', '000000190', '0', '35843', '34353']


In [12]:
config_data['columns']

['198801', '1', '103', '100', '000000190', '0', '35843', '34353']

In [13]:
if util.col_header_val(df,config_data)==0:
    print("validation failed")
    # write code to reject the file
else:
    print("col validation passed")
    # write the code to perform further action
    # in the pipleine

column name and column length validation failed
Following File columns are not in the YAML file ['index']
Following YAML columns are not in the file uploaded []
validation failed


In [16]:
df.describe().compute()  

,198801,1,103,100,000000190,0,35843,34353
count,1.136073e+08,1.136073e+08,1.136073e+08,1.136073e+08,1.136073e+08,1.136073e+08,1.136073e+08,1.136073e+08
mean,2.005126e+05,1.391867e+00,1.932035e+02,3.133187e+02,6.545763e+08,4.707676e+04,1.271670e+05,3.240830e+04
std,9.286563e+02,4.881672e-01,1.210249e+02,1.797740e+02,2.521763e+08,2.914592e+07,4.571867e+06,3.769082e+05
min,1.988010e+05,1.000000e+00,1.030000e+02,1.000000e+02,1.100000e+01,0.000000e+00,0.000000e+00,3.000000e+01
25%,1.997100e+05,1.000000e+00,1.110000e+02,1.040000e+02,4.820109e+08,0.000000e+00,1.160000e+02,7.640000e+02
50%,2.005100e+05,1.000000e+00,2.050000e+02,3.050000e+02,8.306290e+08,0.000000e+00,9.000000e+02,2.782000e+03
75%,2.013080e+05,2.000000e+00,3.020000e+02,5.000000e+02,8.526920e+08,2.600000e+01,8.975000e+03,1.186500e+04
max,2.020120e+05,2.000000e+00,7.030000e+02,9.080000e+02,9.706000e+08,1.255000e+11,1.885790e+09,1.832784e+08


Dask dataframe object has no attribute 'reindex'.<br>
Pandas framewwork could not load dataframe as size was too large and could not allocate memory space.<br>
Thus, index was reset and columns do not match even though the files being compared are the same.

File was downloaded from Kaggle and its size is 4.23 GB.